# Result Analysis

In [ ]:
import os
import cv2
import numpy as np
import lpips
import torch
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
imgs_dir = os.path.join(os.getcwd(), 'data', 'bob_clarens')
number_of_imgs = 200

In [ ]:
def compute_fft(image):
    # Apply FFT
    f = np.fft.fft2(image)
    # Shift the zero frequency component to the center
    fshift = np.fft.fftshift(f)
    # Compute the magnitude spectrum
    magnitude_spectrum = np.abs(fshift)
    return magnitude_spectrum.astype(np.float32)

# MAE

In [ ]:
def calculate_mae(image1, image2):
    # Ensure the images have the same dimensions
    assert image1.shape == image2.shape, "Images must have the same dimensions"
    
    # Compute the Mean Absolute Error
    mae = np.mean(np.abs(image1 - image2))
    return mae

## Image Domain

In [ ]:
mae_1_lst = np.array([])
mae_2_lst = np.array([])
mae_3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    mae_1 = calculate_mae(ref, gsir)
    mae_2 = calculate_mae(ref, tensoir)
    mae_3 = calculate_mae(ref, mine)

    mae_1_lst = np.append(mae_1_lst, mae_1)
    mae_2_lst = np.append(mae_2_lst, mae_2)
    mae_3_lst = np.append(mae_3_lst, mae_3)
        

print(f'MAE between Reference and GS-IR: {mae_1_lst.mean()}')
print(f'MAE between Reference and TensoIR: {mae_2_lst.mean()}')
print(f'MAE between Reference and Mine: {mae_3_lst.mean()}')

## Fourier Domain

In [ ]:
mae_1_lst = np.array([])
mae_2_lst = np.array([])
mae_3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    mine_spectrum = compute_fft(mine_gray)

    mae_1 = calculate_mae(ref, gsir)
    mae_2 = calculate_mae(ref, tensoir)
    mae_3 = calculate_mae(ref, mine)

    mae_1_lst = np.append(mae_1_lst, mae_1)
    mae_2_lst = np.append(mae_2_lst, mae_2)
    mae_3_lst = np.append(mae_3_lst, mae_3)

print(f'MAE between Reference and GS-IR: {mae_1_lst.mean()}')
print(f'MAE between Reference and TensoIR: {mae_2_lst.mean()}')
print(f'MAE between Reference and Mine: {mae_3_lst.mean()}')

# MSE

In [ ]:
def mse(imageA, imageB):
    # Compute the Mean Squared Error between the two images
    return np.mean((imageA - imageB) ** 2)

## Image Domain

In [ ]:
mse_1_lst = np.array([])
mse_2_lst = np.array([])
mse_3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    mse_1 = mse(ref, gsir)
    mse_2 = mse(ref, tensoir)
    mse_3 = mse(ref, mine)

    mse_1_lst = np.append(mse_1_lst, mse_1)
    mse_2_lst = np.append(mse_2_lst, mse_2)
    mse_3_lst = np.append(mse_3_lst, mse_3)

print(f'MSE between Reference and GS-IR: {mse_1_lst.mean()}')
print(f'MSE between Reference and TensoIR: {mse_2_lst.mean()}')
print(f'MSE between Reference and Mine: {mse_3_lst.mean()}')

## Fourier Domain

In [ ]:
# Compute MSE between the reference spectrum and each reconstruction spectrum
mse_1_lst = np.array([])
mse_2_lst = np.array([])
mse_3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    mine_spectrum = compute_fft(mine_gray)

    mse_1 = mse(ref_spectrum, gsir_spectrum)
    mse_2 = mse(ref_spectrum, tensoir_spectrum)
    mse_3 = mse(ref_spectrum, mine_spectrum)

    mse_1_lst = np.append(mse_1_lst, mse_1)
    mse_2_lst = np.append(mse_2_lst, mse_2)
    mse_3_lst = np.append(mse_3_lst, mse_3)

print('MSE between Reference and GS-IR:', mse_1_lst.mean())
print('MSE between Reference and TensoIR:', mse_2_lst.mean())
print('MSE between Reference and Mine:', mse_3_lst.mean())

# SSIM

## Image Domain

In [ ]:
ssim_score1_lst = np.array([])
ssim_score2_lst = np.array([])
ssim_score3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    data_range = ref.max() - ref.min()
    ssim_score1, ssim_image1 = ssim(ref, gsir, data_range=data_range, full=True, win_size=11, channel_axis=2)
    ssim_score2, ssim_image2 = ssim(ref, tensoir, data_range=data_range, full=True, win_size=11, channel_axis=2)
    ssim_score3, ssim_image3 = ssim(ref, mine, data_range=data_range, full=True, win_size=11, channel_axis=2)

    ssim_score1_lst = np.append(ssim_score1_lst, ssim_score1)
    ssim_score2_lst = np.append(ssim_score2_lst, ssim_score2)
    ssim_score3_lst = np.append(ssim_score3_lst, ssim_score3)

print(f'SSIM between Reference and GS-IR: {ssim_score1_lst.mean()}')
print(f'SSIM between Reference and TensoIR: {ssim_score2_lst.mean()}')
print(f'SSIM between Reference and Mine: {ssim_score3_lst.mean()}')

## Fourier Domain

In [ ]:
ssim_score1_lst = np.array([])
ssim_score2_lst = np.array([])
ssim_score3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    mine_spectrum = compute_fft(mine_gray)

    # Compute SSIM
    data_range = ref_spectrum.max() - ref_spectrum.min()
    ssim_score1, ssim_image1 = ssim(ref_spectrum, gsir_spectrum, full=True, data_range=data_range)
    ssim_score2, ssim_image2 = ssim(ref_spectrum, tensoir_spectrum, full=True, data_range=data_range)
    ssim_score3, ssim_image3 = ssim(ref_spectrum, mine_spectrum, full=True, data_range=data_range)

    ssim_score1_lst = np.append(ssim_score1_lst, ssim_score1)
    ssim_score2_lst = np.append(ssim_score2_lst, ssim_score2)
    ssim_score3_lst = np.append(ssim_score3_lst, ssim_score3)

print('SSIM between Reference and GS-IR:', ssim_score1_lst.mean())
print('SSIM between Reference and TensoIR:', ssim_score2.mean())
print('SSIM between Reference and Mine:', ssim_score3.mean()) 

# PSNR

## Image Domain

In [ ]:
psnr_score1_lst = np.array([])
psnr_score2_lst = np.array([])
psnr_score3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    data_range = ref.max() - ref.min()
    psnr_score1 = psnr(ref, gsir, data_range=data_range)
    psnr_score2 = psnr(ref, tensoir, data_range=data_range)
    psnr_score3 = psnr(ref, mine, data_range=data_range)

    psnr_score1_lst = np.append(psnr_score1_lst, psnr_score1)
    psnr_score2_lst = np.append(psnr_score2_lst, psnr_score2)
    psnr_score3_lst = np.append(psnr_score3_lst, psnr_score3)

print(f'PSNR between Reference and GS-IR: {psnr_score1_lst.mean()}')
print(f'PSNR between Reference and TensoIR: {psnr_score2_lst.mean()}')
print(f'PSNR between Reference and Mine: {psnr_score3_lst.mean()}')

## Fourier Domain

In [ ]:
psnr_score1_lst = np.array([])
psnr_score2_lst = np.array([])
psnr_score3_lst = np.array([])

for i in range(number_of_imgs):
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    mine_spectrum = compute_fft(mine_gray)

    data_range = ref_spectrum.max() - ref_spectrum.min()
    psnr_value1 = psnr(ref_spectrum, gsir_spectrum, data_range=data_range)
    psnr_value2 = psnr(ref_spectrum, tensoir_spectrum, data_range=data_range)
    psnr_value3 = psnr(ref_spectrum, mine_spectrum, data_range=data_range)

    psnr_score1_lst = np.append(psnr_score1_lst, psnr_value1)
    psnr_score2_lst = np.append(psnr_score2_lst, psnr_value2)
    psnr_score3_lst = np.append(psnr_score3_lst, psnr_value3)

print('PSNR between Reference and GS-IR:', psnr_score1_lst.mean())
print('PSNR between Reference and TensoIR:', psnr_score2_lst.mean())
print('PSNR between Reference and Mine:', psnr_score3_lst.mean())

# LPIPS

In [ ]:
# Initialize LPIPS model
# You can use 'alex', 'vgg', or 'squeeze'
lpips_model_alex = lpips.LPIPS(net='alex') 
lpips_model_vgg = lpips.LPIPS(net='vgg') 
lpips_model_squeeze = lpips.LPIPS(net='squeeze') 

## Image Domain

In [ ]:
ref_imgs = []
gsir_imgs = []
tensoir_imgs = []
mine_imgs = []

for i in range(number_of_imgs):
    print(f'Processing image {i}...')
    # Load the images as RGB
    ref = np.array(Image.open(os.path.join(imgs_dir, 'ref', f'r_{i}.png')).convert('RGB'))
    gsir = np.array(Image.open(os.path.join(imgs_dir, 'gsir', f'r_{i}.png')).convert('RGB'))
    tensoir = np.array(Image.open(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png')).convert('RGB'))
    mine = np.array(Image.open(os.path.join(imgs_dir, 'mine', f'r_{i}.png')).convert('RGB'))

    ref_tensor = torch.from_numpy(ref).permute(2, 0, 1).unsqueeze(0)
    gsir_tensor = torch.from_numpy(gsir).permute(2, 0, 1).unsqueeze(0)
    tensoir_tensor = torch.from_numpy(tensoir).permute(2, 0, 1).unsqueeze(0)
    mine_tensor = torch.from_numpy(mine).permute(2, 0, 1).unsqueeze(0)

    ref_imgs.append(ref_tensor)
    gsir_imgs.append(gsir_tensor)
    tensoir_imgs.append(tensoir_tensor)
    mine_imgs.append(mine_tensor)

In [ ]:
ref_imgs = torch.cat(ref_imgs, dim=0)
gsir_imgs = torch.cat(gsir_imgs, dim=0)
tensoir_imgs = torch.cat(tensoir_imgs, dim=0)
mine_imgs = torch.cat(mine_imgs, dim=0)

### Alex

In [ ]:
# Compute LPIPS
lpips_alex_score1 = lpips_model_alex(ref_imgs, gsir_imgs)
lpips_alex_score2 = lpips_model_alex(ref_imgs, tensoir_imgs)
lpips_alex_score3 = lpips_model_alex(ref_imgs, mine_imgs)

In [ ]:
lpips_alex_score1_rgb_mean = torch.mean(lpips_alex_score1)
lpips_alex_score2_rgb_mean = torch.mean(lpips_alex_score2)
lpips_alex_score3_rgb_mean = torch.mean(lpips_alex_score3)

print('LPIPS distance between Reference and GS-IR:', lpips_alex_score1_rgb_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_alex_score2_rgb_mean)
print('LPIPS distance between Reference and Mine:', lpips_alex_score3_rgb_mean)

### VGG

In [ ]:
lpips_vgg_score1 = lpips_model_vgg(ref_imgs, gsir_imgs)
lpips_vgg_score2 = lpips_model_vgg(ref_imgs, tensoir_imgs)
lpips_vgg_score3 = lpips_model_vgg(ref_imgs, mine_imgs)

In [ ]:
lpips_vgg_score1_rgb_mean = torch.mean(lpips_vgg_score1)
lpips_vgg_score2_rgb_mean = torch.mean(lpips_vgg_score2)
lpips_vgg_score3_rgb_mean = torch.mean(lpips_vgg_score3)

print('LPIPS distance between Reference and GS-IR:', lpips_vgg_score1_rgb_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_vgg_score2_rgb_mean)
print('LPIPS distance between Reference and Mine:', lpips_vgg_score3_rgb_mean)

### Squeeze

In [ ]:
lpips_squeeze_score1 = lpips_model_squeeze(ref_imgs, gsir_imgs)
lpips_squeeze_score2 = lpips_model_squeeze(ref_imgs, tensoir_imgs)
lpips_squeeze_score3 = lpips_model_squeeze(ref_imgs, mine_imgs)

In [ ]:
lpips_squeeze_score1_rgb_mean = torch.mean(lpips_squeeze_score1)
lpips_squeeze_score2_rgb_mean = torch.mean(lpips_squeeze_score2)
lpips_squeeze_score3_rgb_mean = torch.mean(lpips_squeeze_score3)

print('LPIPS distance between Reference and GS-IR:', lpips_squeeze_score1_rgb_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_squeeze_score2_rgb_mean)
print('LPIPS distance between Reference and Mine:', lpips_squeeze_score3_rgb_mean)

## Fourier Domain

In [ ]:
ref_imgs = []
gsir_imgs = []
tensoir_imgs = []
mine_imgs = []

for i in range(number_of_imgs):
    print(f'Processing image {i}...')
    # Load the images as grayscale
    ref_gray = cv2.imread(os.path.join(imgs_dir, 'ref', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    gsir_gray = cv2.imread(os.path.join(imgs_dir, 'gsir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    tensoir_gray = cv2.imread(os.path.join(imgs_dir, 'tensoir', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)
    mine_gray = cv2.imread(os.path.join(imgs_dir, 'mine', f'r_{i}.png'), cv2.IMREAD_GRAYSCALE)

    ref_spectrum = compute_fft(ref_gray)
    gsir_spectrum = compute_fft(gsir_gray)
    tensoir_spectrum = compute_fft(tensoir_gray)
    mine_spectrum = compute_fft(mine_gray)

    ref_spectrum_tensor = torch.from_numpy(ref_spectrum).unsqueeze(0)
    gsir_spectrum_tensor = torch.from_numpy(gsir_spectrum).unsqueeze(0)
    tensoir_spectrum_tensor = torch.from_numpy(tensoir_spectrum).unsqueeze(0)
    mine_spectrum_tensor = torch.from_numpy(mine_spectrum).unsqueeze(0)

    ref_imgs.append(ref_spectrum_tensor)
    gsir_imgs.append(gsir_spectrum_tensor)
    tensoir_imgs.append(tensoir_spectrum_tensor)
    mine_imgs.append(mine_spectrum_tensor)

In [ ]:
ref_imgs = torch.cat(ref_imgs, dim=0)
gsir_imgs = torch.cat(gsir_imgs, dim=0)
tensoir_imgs = torch.cat(tensoir_imgs, dim=0)
mine_imgs = torch.cat(mine_imgs, dim=0)

### Alex

In [ ]:
lpips_alex_score1 = lpips_model_alex(ref_spectrum_tensor, gsir_spectrum_tensor)
lpips_alex_score2 = lpips_model_alex(ref_spectrum_tensor, tensoir_spectrum_tensor)
lpips_alex_score3 = lpips_model_alex(ref_spectrum_tensor, mine_spectrum_tensor)

In [ ]:
lpips_alex_score1_fourier_mean = torch.mean(lpips_alex_score1)
lpips_alex_score2_fourier_mean = torch.mean(lpips_alex_score2)
lpips_alex_score3_fourier_mean = torch.mean(lpips_alex_score3)

print('LPIPS distance between Reference and GS-IR:', lpips_alex_score1_fourier_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_alex_score2_fourier_mean)
print('LPIPS distance between Reference and Mine:', lpips_alex_score3_fourier_mean)

### VGG

In [ ]:
lpips_vgg_score1 = lpips_model_vgg(ref_spectrum_tensor, gsir_spectrum_tensor)
lpips_vgg_score2 = lpips_model_vgg(ref_spectrum_tensor, tensoir_spectrum_tensor)
lpips_vgg_score3 = lpips_model_vgg(ref_spectrum_tensor, mine_spectrum_tensor)

In [ ]:
lpips_vgg_score1_fourier_mean = torch.mean(lpips_vgg_score1)
lpips_vgg_score2_fourier_mean = torch.mean(lpips_vgg_score2)
lpips_vgg_score3_fourier_mean = torch.mean(lpips_vgg_score3)

print('LPIPS distance between Reference and GS-IR:', lpips_vgg_score1_fourier_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_vgg_score2_fourier_mean)
print('LPIPS distance between Reference and Mine:', lpips_vgg_score3_fourier_mean)

### Squeeze

In [ ]:
lpips_squeeze_score1 = lpips_model_squeeze(ref_spectrum_tensor, gsir_spectrum_tensor)
lpips_squeeze_score2 = lpips_model_squeeze(ref_spectrum_tensor, tensoir_spectrum_tensor)
lpips_squeeze_score3 = lpips_model_squeeze(ref_spectrum_tensor, mine_spectrum_tensor)

In [ ]:
lpips_squeeze_score1_fourier_mean = torch.mean(lpips_squeeze_score1)
lpips_squeeze_score2_fourier_mean = torch.mean(lpips_squeeze_score2)
lpips_squeeze_score3_fourier_mean = torch.mean(lpips_squeeze_score3)

print('LPIPS distance between Reference and GS-IR:', lpips_squeeze_score1_fourier_mean)
print('LPIPS distance between Reference and TensoIR:', lpips_squeeze_score2_fourier_mean)
print('LPIPS distance between Reference and Mine:', lpips_squeeze_score3_fourier_mean)

## FLIP

Check Windows